# Day 6 Lab 2: Deploying & Using Existing Endpoints

## 🎯 Learning Objectives
- Deploy a pre-trained model endpoint
- Invoke endpoint from Python, CLI, Lambda
- Implement error handling and retry logic
- Monitor endpoint metrics

## 🏦 Banking Use Case
Deploy a **fraud detection model** and integrate it with multiple banking applications.

## ⏱️ Duration: 35 minutes
## 💰 Cost: ~$0.12

## Setup

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn import SKLearnModel
import json
import time
from botocore.exceptions import ClientError

# Initialize
session = sagemaker.Session()
role = get_execution_role()
region = session.boto_region_name
bucket = session.default_bucket()
runtime = boto3.client('sagemaker-runtime', region_name=region)

print(f"Region: {region}")
print(f"Bucket: {bucket}")

## Part 1: Deploy Fraud Detection Endpoint

In [ ]:
# Use pre-trained fraud detection model
model_data = f"s3://{bucket}/sagemaker/fraud-detection-model/model.tar.gz"

sklearn_model = SKLearnModel(
    model_data=model_data,
    role=role,
    entry_point='inference.py',
    framework_version='1.0-1',
    py_version='py3'
)

endpoint_name = f"fraud-detection-{int(time.time())}"
print(f"Deploying endpoint: {endpoint_name}")

predictor = sklearn_model.deploy(
    instance_type='ml.m5.large',
    initial_instance_count=1,
    endpoint_name=endpoint_name
)

print(f"✅ Endpoint deployed: {endpoint_name}")
print(f"Status: InService")

## Part 2: Invoke from Python (Boto3)

**Most common method for production applications**

In [ ]:
# Sample transaction data
transaction = {
    'amount': 5000.00,
    'merchant_category': 'electronics',
    'location': 'international',
    'time_of_day': 'night',
    'card_present': False
}

# Invoke endpoint
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=json.dumps(transaction)
)

# Parse response
result = json.loads(response['Body'].read().decode())
print(f"\n🔍 Fraud Detection Result:")
print(f"  Transaction: ${transaction['amount']}")
print(f"  Fraud Probability: {result['fraud_probability']:.2%}")
print(f"  Decision: {result['decision']}")
print(f"  Reason: {result['reason']}")

## Part 3: Batch Processing with Python

In [ ]:
# Batch of transactions
transactions = [
    {'amount': 50, 'merchant_category': 'grocery', 'location': 'local', 'time_of_day': 'day', 'card_present': True},
    {'amount': 10000, 'merchant_category': 'jewelry', 'location': 'international', 'time_of_day': 'night', 'card_present': False},
    {'amount': 200, 'merchant_category': 'restaurant', 'location': 'local', 'time_of_day': 'evening', 'card_present': True},
    {'amount': 7500, 'merchant_category': 'electronics', 'location': 'international', 'time_of_day': 'night', 'card_present': False}
]

print("Processing batch of transactions...\n")
flagged_count = 0

for i, txn in enumerate(transactions, 1):
    response = runtime.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='application/json',
        Body=json.dumps(txn)
    )
    result = json.loads(response['Body'].read().decode())
    
    print(f"Transaction {i}: ${txn['amount']} - {result['decision']}")
    if result['decision'] == 'FLAGGED':
        flagged_count += 1

print(f"\n📊 Summary: {flagged_count}/{len(transactions)} transactions flagged for review")

## Part 4: Invoke from AWS CLI

**Useful for testing and shell scripts**

In [ ]:
# Create input file
test_input = {
    'amount': 3000,
    'merchant_category': 'travel',
    'location': 'international',
    'time_of_day': 'morning',
    'card_present': False
}

with open('/tmp/input.json', 'w') as f:
    json.dump(test_input, f)

# CLI command
cli_command = f"""
aws sagemaker-runtime invoke-endpoint \\
    --endpoint-name {endpoint_name} \\
    --content-type application/json \\
    --body file:///tmp/input.json \\
    /tmp/output.json
"""

print("📝 AWS CLI Command:")
print(cli_command)
print("\n💡 Use this command in shell scripts or CI/CD pipelines")

## Part 5: Error Handling & Retry Logic

**Production-grade implementation**

In [ ]:
import time
from botocore.exceptions import ClientError

def invoke_endpoint_with_retry(endpoint_name, payload, max_retries=3):
    """
    Invoke SageMaker endpoint with exponential backoff retry logic
    """
    for attempt in range(max_retries):
        try:
            response = runtime.invoke_endpoint(
                EndpointName=endpoint_name,
                ContentType='application/json',
                Body=json.dumps(payload)
            )
            return json.loads(response['Body'].read().decode())
            
        except ClientError as e:
            error_code = e.response['Error']['Code']
            
            if error_code == 'ThrottlingException':
                # Exponential backoff
                wait_time = (2 ** attempt) + (time.time() % 1)
                print(f"⚠️ Throttled. Retrying in {wait_time:.2f}s...")
                time.sleep(wait_time)
                
            elif error_code == 'ModelError':
                print(f"❌ Model error: {e}")
                raise
                
            elif error_code == 'ValidationError':
                print(f"❌ Invalid input: {e}")
                raise
                
            else:
                print(f"❌ Unexpected error: {e}")
                if attempt == max_retries - 1:
                    raise
                time.sleep(2 ** attempt)
    
    raise Exception(f"Failed after {max_retries} retries")

# Test retry logic
test_transaction = {
    'amount': 1500,
    'merchant_category': 'retail',
    'location': 'local',
    'time_of_day': 'afternoon',
    'card_present': True
}

print("Testing retry logic...")
result = invoke_endpoint_with_retry(endpoint_name, test_transaction)
print(f"✅ Success: {result['decision']}")

## Part 6: Monitor Endpoint Metrics

In [ ]:
cloudwatch = boto3.client('cloudwatch', region_name=region)

def get_endpoint_metrics(endpoint_name):
    metrics = {}
    
    for metric_name in ['ModelLatency', 'Invocations', 'Invocation4XXErrors', 'Invocation5XXErrors']:
        response = cloudwatch.get_metric_statistics(
            Namespace='AWS/SageMaker',
            MetricName=metric_name,
            Dimensions=[{'Name': 'EndpointName', 'Value': endpoint_name}],
            StartTime=time.time() - 600,
            EndTime=time.time(),
            Period=300,
            Statistics=['Average', 'Sum']
        )
        
        if response['Datapoints']:
            datapoint = response['Datapoints'][0]
            metrics[metric_name] = datapoint.get('Average') or datapoint.get('Sum', 0)
        else:
            metrics[metric_name] = 0
    
    return metrics

# Get metrics
print("📈 Endpoint Metrics (Last 10 minutes):\n")
metrics = get_endpoint_metrics(endpoint_name)

print(f"  Model Latency: {metrics['ModelLatency']:.2f}ms")
print(f"  Total Invocations: {int(metrics['Invocations'])}")
print(f"  4XX Errors: {int(metrics['Invocation4XXErrors'])}")
print(f"  5XX Errors: {int(metrics['Invocation5XXErrors'])}")

# Calculate success rate
total = metrics['Invocations']
errors = metrics['Invocation4XXErrors'] + metrics['Invocation5XXErrors']
success_rate = ((total - errors) / total * 100) if total > 0 else 100

print(f"\n  Success Rate: {success_rate:.2f}%")
print(f"\n💡 Target: >99.9% success rate, <100ms latency")

## Part 7: Lambda Integration (Conceptual)

**How to integrate with Lambda for API Gateway**

In [ ]:
# Lambda function code (for reference)
lambda_code = f"""
import json
import boto3

runtime = boto3.client('sagemaker-runtime')

def lambda_handler(event, context):
    # Parse transaction from API Gateway
    transaction = json.loads(event['body'])
    
    # Invoke SageMaker endpoint
    response = runtime.invoke_endpoint(
        EndpointName='{endpoint_name}',
        ContentType='application/json',
        Body=json.dumps(transaction)
    )
    
    # Parse result
    result = json.loads(response['Body'].read().decode())
    
    # Return to API Gateway
    return {{
        'statusCode': 200,
        'body': json.dumps(result),
        'headers': {{
            'Content-Type': 'application/json',
            'Access-Control-Allow-Origin': '*'
        }}
    }}
"""

print("📝 Lambda Function Code:")
print(lambda_code)
print("\n💡 Architecture: API Gateway → Lambda → SageMaker Endpoint")
print("\n✅ Benefits:")
print("  - Serverless scaling")
print("  - Pay per request")
print("  - Easy API management")
print("  - Built-in authentication")

## Part 8: IAM Permissions Required

In [ ]:
# IAM policy for endpoint invocation
iam_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sagemaker:InvokeEndpoint"
            ],
            "Resource": f"arn:aws:sagemaker:{region}:*:endpoint/{endpoint_name}"
        }
    ]
}

print("📝 Required IAM Policy:")
print(json.dumps(iam_policy, indent=2))
print("\n💡 Attach this policy to:")
print("  - Lambda execution role")
print("  - EC2 instance role")
print("  - Application service account")
print("\n✅ Principle of least privilege: Only InvokeEndpoint permission")

## Cleanup

In [ ]:
# Delete endpoint
print("Cleaning up endpoint...")
predictor.delete_endpoint()
print("✅ Cleanup complete")

## 🎓 Key Takeaways

1. **Invocation Methods:**
   - ✅ Python (Boto3): Most common, production apps
   - ✅ AWS CLI: Testing, shell scripts
   - ✅ Lambda: Serverless integration
   - ✅ REST API: Web applications

2. **Error Handling:**
   - ✅ Implement retry logic with exponential backoff
   - ✅ Handle ThrottlingException
   - ✅ Handle ModelError and ValidationError
   - ✅ Set appropriate timeouts

3. **Monitoring:**
   - ✅ Track ModelLatency (target: <100ms)
   - ✅ Monitor Invocations count
   - ✅ Watch for 4XX/5XX errors
   - ✅ Set CloudWatch alarms

4. **IAM Permissions:**
   - ✅ sagemaker:InvokeEndpoint required
   - ✅ Use least-privilege principle
   - ✅ Resource-specific policies

5. **Production Best Practices:**
   - ✅ Always implement retry logic
   - ✅ Monitor metrics continuously
   - ✅ Use Lambda for serverless scaling
   - ✅ Set up CloudWatch alarms
   - ✅ Test error scenarios